In [2228]:
import csv
import re
import emoji
import os
import pickle
from datetime import datetime
import logging
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertForSequenceClassification, AdamW, BertConfig
from tqdm import tqdm, trange
import numpy as np
import time
import random
import datetime
from _datetime import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
import pandas as pd
import math
from transformers import AutoModel, AutoTokenizer, AutoModelWithLMHead
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForMaskedLM,
    BertTokenizer,
    CamembertConfig,
    CamembertForMaskedLM,
    CamembertTokenizer,
    DistilBertConfig,
    DistilBertForMaskedLM,
    DistilBertTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTConfig,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
os.getcwd()

'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\most updated files'

In [2229]:
os.chdir('C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\most updated files')

In [2230]:
os.getcwd()

'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\most updated files'

In [2231]:
# Logger stuff
logFormatter = logging.Formatter("%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
logger = logging.getLogger(__name__)

In [2232]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dani1.OFER3090PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2233]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dani1.OFER3090PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2234]:
stop_words = stopwords.words("english")
wordnet = WordNetLemmatizer()

In [2235]:
def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [2236]:
def load_train_test_data(tokenizer,train_file_path,test_file_path):

    '''
    Function to load training and test data. The input format is one tweet per line:
    tweet_id \t tweet text \t OFF/NOT
    :param language: the tweets language, it is used just for paths, can be removed
    :param tokenizer: BERT tokenizer, output of the training code
    :return: the list of
        train_input_ids, train_labels, train_attention_masks,test_input_ids, test_labels, test_attention_masks
        which stand for tokenized tweet texts, labels and computed attention mask for training and test data respectively
    '''
#     train_file_path = '/path2dataset/dataset_train.csv'
#     test_file_path = '/path2dataset/dataset_test.csv'


    # List of all tweets text
    train_tweets = []
    # List of all labels
    train_labels = []

    # List of all tweets text
    test_tweets = []
    # List of all labels
    test_labels = []

    # -----------------------------------------------------------------
    # Parse Training Set
    with open(train_file_path, encoding="utf8") as input_file:
        # For each tweet
        count = 0
        for line in csv.reader(input_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_ALL):
            if line[0] != 'id' and len(line) == 3:
                full_line = line[1]
                full_line = text_preproc(full_line)
#                 full_line = re.sub(r'#([^ ]*)', r'\1', full_line)
#                 full_line = re.sub(r'https.*[^ ]', 'URL', full_line)
#                 full_line = re.sub(r'http.*[^ ]', 'URL', full_line)
#                 full_line = emoji.demojize(full_line)
#                 full_line = re.sub(r'(:.*?:)', r' \1 ', full_line)
#                 full_line = re.sub(' +', ' ', full_line)
                
                label = line[2]

#                 if line[2] == 'OFF':
#                     label = 1
#                 else:
#                     label = 0

                # Save tweet's text and label
                train_tweets.append(full_line)
                train_labels.append(label)

    # -----------------------------------------------------------------
    # Parse Test Set
    with open(test_file_path, encoding="utf8") as input_file:
        # For each tweet
        for line in csv.reader(input_file, delimiter=","):
            if line[0] != 'id' and len(line) == 3:
                full_line = line[1]
                full_line = text_preproc(full_line)
#                 full_line = re.sub(r'#([^ ]*)', r'\1', full_line)
#                 full_line = re.sub(r'https.*[^ ]', 'URL', full_line)
#                 full_line = re.sub(r'http.*[^ ]', 'URL', full_line)
#                 full_line = emoji.demojize(full_line)
#                 full_line = re.sub(r'(:.*?:)', r' \1 ', full_line)
#                 full_line = re.sub(' +', ' ', full_line)
                label = line[2]
#                 if line[2] == 'OFF':
#                     label = 1
#                 else:
#                     label = 0

                # Save tweet's text and label
                test_tweets.append(full_line)
                test_labels.append(label)

    # List of all tokenized tweets
    train_input_ids = []
    test_input_ids = []

    # For every tweet in the training set
    for sent in train_tweets:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=100, # orig value = 512 - modfied Tommaso 20200408

            # This function also supports truncation and conversion
            # to pytorch tensors, but we need to do padding, so we
            # can't use these features :( .
            # max_length = 128,          # Truncate all sentences.
            # return_tensors = 'pt',     # Return pytorch tensors.
        )

        # Add the encoded tweet to the list.
        train_input_ids.append(encoded_sent)

    # For every tweet in the test set
    for sent in test_tweets:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=100, # orig value = 512 - modfied Tommaso 20200408

            # This function also supports truncation and conversion
            # to pytorch tensors, but we need to do padding, so we
            # can't use these features :( .
            # max_length = 128,          # Truncate all sentences.
            # return_tensors = 'pt',     # Return pytorch tensors.
        )

        # Add the encoded tweet to the list.
        test_input_ids.append(encoded_sent)

    # # Pad our input tokens with value 0.
    # # "post" indicates that we want to pad and truncate at the end of the sequence,
    # # as opposed to the beginning.

    train_input_ids = pad_sequences(train_input_ids, maxlen=100, dtype="long",
                          value=tokenizer.pad_token_id, truncating="pre", padding="pre")

    test_input_ids = pad_sequences(test_input_ids, maxlen=100, dtype="long",
                                    value=tokenizer.pad_token_id, truncating="pre", padding="pre")
    # Create attention masks
    # The attention mask simply makes it explicit which tokens are actual words versus which are padding
    train_attention_masks = []
    test_attention_masks = []

    # For each tweet in the training set
    for sent in train_input_ids:
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        train_attention_masks.append(att_mask)

    # For each tweet in the test set
    for sent in test_input_ids:
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        test_attention_masks.append(att_mask)

    # Return the list of encoded tweets, the list of labels and the list of attention masks
    return train_input_ids, train_labels, train_attention_masks, test_input_ids, test_labels, test_attention_masks

In [2237]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [2238]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [2239]:
# Input language, is used just for paths
#language = 'tr'

# Directory where the pre-trained model can be found (after pre-traing from Huggingface)
#C:\Users\Dani1.OFER3090PC\Documents\Tal Folder\thesis\updated files\HateBert_models\hate_bert
#../input/hatebert-osf/googledrive-archive/retrained_model/hate_bert/hate_bert
model_dir = 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\hate_bert\\'
# Returns a datetime object containing the local date and time (used for output_model_dir)
dateTimeObj = str(dt.now()).replace(" ", "_")
# Directory in which the model will be saved along with the log
#output_model_dir = 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\path2store_retrained_finetuned_model\\' + dateTimeObj + "\\" # "../out/" + language + '/models/' + dateTimeObj + "/"
#output_model_dir = 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\path2store_retrained_finetuned_model\\'
output_model_dir = 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\HateBert_models\\RoastMe_Balanced\\1\\'
# Make dir for model serializations
os.makedirs(os.path.dirname(output_model_dir), exist_ok=True)

In [2240]:
# Log stuff: print logger on file in output_model_dir/log.log
logging.basicConfig(filename=output_model_dir + 'log.log', level=logging.DEBUG)

# Log stuff: print logger also on stderr
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
logger.addHandler(consoleHandler)

In [2241]:
# -----------------------------
# Load Pre-trained BERT model
# -----------------------------

In [2242]:
config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load a trained model and vocabulary pre-trained for specific language
logger.info("Loading model") #from: '" + model_dir + "', it may take a while...")

# Load pre-trained Tokenizer from directory, change this to load a tokenizer from ber package
#tokenizer = tokenizer_class.from_pretrained(model_dir)
tokenizer = tokenizer_class.from_pretrained('GroNLP/hateBERT')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Load Bert for classification 'container'
model = BertForSequenceClassification.from_pretrained(
     model_dir, # Use pre-trained model from its directory, change this to use a pre-trained model from bert
     num_labels = 2, # The number of output labels--2 for binary classification.
                     # You can increase this for multi-class tasks.
     output_attentions = False, # Whether the model returns attentions weights.
     output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Set the model to work on CPU if no GPU is present
model.to(device)
logger.info("HateBERT for classification model has been loaded!")

2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-14 13:46:07,619 [MainThread  ] [INFO ]  Loading model
2022-09-

In [2243]:
# The list of prints can be safely removed

# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

logger.info('The HateBERT model has {:} different named parameters.\n'.format(len(params)))

logger.info('==== Embedding Layer ====\n')

for p in params[0:5]:
    logger.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

logger.info('\n==== First Transformer ====\n')

for p in params[5:21]:
    logger.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

logger.info('\n==== Output Layer ====\n')

for p in params[-4:]:
    logger.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT model has 201 different named parameters.

2022-09-14 13:46:13,903 [MainThread  ] [INFO ]  The HateBERT mod

In [2244]:
def create_balanced_datasets(max_size,dataset_label_0,dataset_label_1):
    #if len(dataset_label_0) + len(dataset_label_1) >  2*max_size:
        minority_len = min(len(dataset_label_0),len(dataset_label_1))
        if  minority_len > max_size:
            balanced_0 = dataset_label_0.sample(max_size)
            balanced_1 = dataset_label_1.sample(max_size)
        else:
            balanced_0 = dataset_label_0.sample(minority_len)
            balanced_1 = dataset_label_1.sample(minority_len)
        balanced_dataset = pd.concat([balanced_0,balanced_1],axis=0,ignore_index=True)
        balanced_dataset = balanced_dataset.sample(frac=1).reset_index(drop=True)
        balanced_dataset['id'] = [i for i in range(1,len(balanced_dataset)+1)]
        #balanced_dataset = shuffle(balanced_dataset)
        return balanced_dataset

--------------------------------------------------------------------
-------------------------- Load test data --------------------------
--------------------------------------------------------------------


In [2245]:
#Roast_Me_dataset  = pd.read_csv('RoastMe_Data_updated.csv',encoding="Latin-1")


In [2246]:
#../input/data-cross-domain/RoastMe_new_Data_balanced.csv
Roast_Me_dataset_new = pd.read_csv('RoastMe_new_Data_balanced.csv',encoding="Latin-1")

In [2247]:
#../input/data-cross-domain/Founta_Data_balanced.csv
Founta_Dataset = pd.read_csv('Founta_Data_balanced.csv')

In [2248]:
#../input/data-cross-domain/Razavi_Data_Balanced.csv
Razavi_Dataset = pd.read_csv('Razavi_Data_balanced.csv')

In [2249]:
Waseem_Dataset = pd.read_csv('Waseem_Data_balanced.csv')

In [2250]:
#../input/data-cross-domain/Kumar_Data_balanced.csv
Kumar_Dataset = pd.read_csv('Kumar_Data_balanced.csv')

In [2251]:
#../input/data-cross-domain/Offensive Reddit Data balanced.csv
Offensive_Reddit_Dataset = pd.read_csv('Offensive_Reddit_new_Data_balanced.csv')

In [2252]:
#../input/data-cross-domain/Sarcastic Twitter Data balanced.csv
Sarcastic_Twitter_Dataset = pd.read_csv('Sarcastic_Twitter_Data_balanced.csv')

In [2253]:
#../input/data-cross-domain/Sarcastic Reddit Data balanced.csv
Sarcastic_Reddit_Dataset = pd.read_csv('Sarcastic_Reddit_Data_balanced.csv')

In [2254]:
#../input/data-cross-domain/
kaggle_dataset = pd.read_csv('Kaggle_Data_balanced.csv')

In [2255]:
#../input/data-cross-domain/Non Offensive Reddit Data balanced.csv
#non_offensive_dataset = pd.read_csv('Non_Offensive_Reddit_Data_balanced.csv')

In [2256]:
#../input/data-cross-domain/Non Offensive Reddit 2 Data balanced.csv
#non_offensive_dataset_2 = pd.read_csv('Non_Offensive_Reddit_2_Data_balanced.csv')

In [2257]:
len(Razavi_Dataset)

964

In [2258]:
# for i in range(5):
#     train_dataset = create_balanced_datasets(1500,Roast_Me_dataset[Roast_Me_dataset['Label']==0],Roast_Me_dataset[Roast_Me_dataset['Label']==1])
#     #train_dataset = kaggle_dataset
#     train_rate = 0.8
#     train_text, val_text, train_labels, val_labels = train_test_split(train_dataset['comment'], train_dataset['Label'], 
#                                                                     #random_state=2018,
#                                                                     test_size= 1-train_rate,
#                                                                     stratify=train_dataset['Label'])
#     train_labels = train_labels.reset_index(drop=True)
#     train_text = train_text.reset_index(drop=True)
#     val_labels = val_labels.reset_index(drop=True)
#     val_text = val_text.reset_index(drop=True)
#     train_dataset_current = pd.DataFrame()
#     train_dataset_current['id'] = [i for i in range(1,len(train_text)+1)]
#     train_dataset_current['comment'] = train_text
#     train_dataset_current['Label'] = train_labels
#     validataion_dataset_current = pd.DataFrame()
#     validataion_dataset_current['id'] = [i for i in range(1,len(val_text)+1)]
#     validataion_dataset_current['comment'] = val_text
#     validataion_dataset_current['Label'] = val_labels
#     train_dataset_current.to_csv('train_dataset_'+ str(i+1) +'.csv',index=False, encoding="utf8" )
#     validataion_dataset_current.to_csv('validation_dataset_'+ str(i+1) +'.csv',index=False, encoding="utf8" )

In [2259]:
def devide_sataset_to_n_train_datasets(train_dataset_full,dataset_name,n):
    #n = 5
    train_dataset_full = train_dataset_full
    train_class_len = math.floor(math.floor(len(train_dataset_full)/n)/2)
    train_current = train_dataset_full
    for i in range(n):
        train_dataset = create_balanced_datasets(train_class_len,train_current[train_current['Label']==0],train_current[train_current['Label']==1])
    #train_dataset = kaggle_dataset
        train_current = pd.merge(train_current,train_dataset, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
        train_rate = 0.8
        train_text, val_text, train_labels, val_labels = train_test_split(train_dataset['comment'], train_dataset['Label'], 
                                                                    #random_state=2018,
                                                                    test_size= 1-train_rate,
                                                                    stratify=train_dataset['Label'])
        train_labels = train_labels.reset_index(drop=True)
        train_text = train_text.reset_index(drop=True)
        val_labels = val_labels.reset_index(drop=True)
        val_text = val_text.reset_index(drop=True)
        train_dataset_current = pd.DataFrame()
        train_dataset_current['id'] = [i for i in range(1,len(train_text)+1)]
        train_dataset_current['comment'] = train_text
        train_dataset_current['Label'] = train_labels
        validataion_dataset_current = pd.DataFrame()
        validataion_dataset_current['id'] = [i for i in range(1,len(val_text)+1)]
        validataion_dataset_current['comment'] = val_text
        validataion_dataset_current['Label'] = val_labels
        train_dataset_current.to_csv('/kaggle/working/train_dataset_' + str(i+1) +'_'+ dataset_name+'.csv',index=False, encoding="utf8" )
        validataion_dataset_current.to_csv('/kaggle/working/validation_dataset_' + str(i+1)+'_' + dataset_name+'.csv',index=False, encoding="utf8" )

In [2260]:
#devide_sataset_to_n_train_datasets(Offensive_Reddit_Dataset,'Offensive_Reddit',3)

In [2261]:
def create_dataset_with_Roastme(train_dataset_name,n):
    RM_train = pd.read_csv('/kaggle/input/data-cross-domain/train_dataset_1_RoastMe.csv')
    RM_train = RM_train.sample(frac=0.85)
    RM_validation = pd.read_csv('/kaggle/input/data-cross-domain/validation_dataset_1_RoastMe.csv')
    RM_validation = RM_validation.sample(frac=0.85)
    for i in range(1,n+1):
        train_dataset = pd.read_csv('train_dataset_'+ str(i)+'_'+train_dataset_name+'.csv')
        train_dataset = pd.concat([train_dataset,RM_train])
        train_dataset = train_dataset.sample(frac=1)
        train_dataset = train_dataset.reset_index(drop=True)
        train_dataset['id'] = range(1,len(train_dataset)+1)
        train_dataset.to_csv('/kaggle/working/train_dataset_'+str(i)+'_'+train_dataset_name + '_RM.csv',index=False)
        validation_dataset = pd.read_csv('validation_dataset_'+ str(i)+'_'+train_dataset_name+'.csv')
        validation_dataset = pd.concat([validation_dataset,RM_validation])
        validation_dataset = validation_dataset.sample(frac=1)
        validation_dataset = validation_dataset.reset_index(drop=True)
        validation_dataset['id'] = range(1,len(validation_dataset)+1)
        validation_dataset.to_csv('/kaggle/working/validation_dataset_'+str(i)+'_'+train_dataset_name + '_RM.csv',index=False)

In [2262]:
def create_train_and_validation_datasets_for_self_classification(dataset_name, validation_rate, numOfDatasets,train_dataset_number):
    for i in range(1,numOfDatasets+1):
        train_dataset = pd.DataFrame()
        validation_dataset = pd.DataFrame()
        path = 'train_dataset_' + str(train_dataset_number) + '_' + dataset_name + '.csv'
        dataset = pd.read_csv(path)
        X_train, X_validation, Y_train, Y_validation = train_test_split(dataset['comment'],dataset['Label'], test_size= validation_rate,
                                                                    stratify=dataset['Label'])
        
        train_dataset['id'] = range(1,len(X_train)+1)
        train_dataset['comment'] = X_train.reset_index(drop=True)
        train_dataset['Label'] = Y_train.reset_index(drop=True)
        validation_dataset['id'] = range(1,len(X_validation)+1)
        validation_dataset['comment'] = X_validation.reset_index(drop=True)
        validation_dataset['Label'] = Y_validation.reset_index(drop=True)
        path_train = 'train_train_dataset_'+str(i) +'_' + dataset_name + '.csv'
        path_validation = 'validation_train_dataset_'+str(i) +'_'+ dataset_name + '.csv'
        train_dataset.to_csv(path_train, index=False)
        validation_dataset.to_csv(path_validation, index=False)

In [2263]:
#create_train_and_validation_datasets_for_self_classification('Waseem',0.2,4,1)

In [2264]:
# The loading eval data return:
# - input_ids:         the list of all tweets already tokenized and ready for bert (with [CLS] and [SEP])
# - labels:            the list of labels, the i-th index corresponds to the i-th position in input_ids
# - attention_masks:   a list of [0,1] for every input_id that represent which token is a padding token and which is not
#../input/data-cross-domain
# ../input/data-cross-domain/train_dataset_1_Kaggle_imbalanced.csv
#../input/data-cross-domain/train_dataset_1_Kaggle_imbalanced.csv
#../input/data-cross-domain/validation_dataset_1_Kaggle_imbalanced.csv
#../input/data-cross-domain/train_train_dataset_1_Razavi_imbalanced.csv
#../input/data-cross-domain/validation_train_dataset_1_Razavi_imbalanced.csv
train_path = 'train_dataset_1_Kaggle.csv'
validation_path = 'validation_dataset_1_Kaggle.csv'
train_inputs, train_labels, train_masks, validation_inputs, validation_labels, validation_masks = load_train_test_data(tokenizer,train_path,validation_path)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [2265]:
integer_map = map(int, train_labels)
train_labels = list(integer_map)
integer_map = map(int, validation_labels)
validation_labels = list(integer_map)
# Tweets
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

# Labels
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

# Attention masks
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
# Note that the number of batch has to be the same, this means that we have to aggregate results in the end
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [2266]:
train_labels

tensor([1, 1, 1,  ..., 1, 1, 0])

# --------------------------------------------------------------------
# -------------- Optimizer and Learning Rate Scheduler ---------------
# --------------------------------------------------------------------

In [2267]:
# For the purposes of fine-tuning, the authors recommend choosing from the following values:

# Batch size: 16, 32 (We chose 32 when creating our DataLoaders).
# Learning rate (Adam): 5e-5, 3e-5, 2e-5 (We’ll use 2e-5).
# Number of epochs: 2, 3, 4 (We’ll use 4).
#
#
# Note: AdamW is a class from the HuggingFace library (as opposed to PyTorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr=1e-5,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps=1e-8  # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs (authors recommend between 2 and 4)
epochs = 5

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps)
# --------------------------------------------------------------------
# Now we are ready to prepare and run the training/evaluation
# --------------------------------------------------------------------
#
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in tqdm(range(0, epochs), desc="Training"):

    # ========================================
    #               Training
    # ========================================

    # Store true lables for global eval
    gold_labels = []
    # Store  predicted labels for global eval
    predicted_labels = []

    # Perform one full pass over the training set.

    logger.info("")
    logger.info('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    logger.info('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    # the tqdm instruction mess with prints on terminal but it can be useful to understand what is the current
    # batch at any time
    for step, batch in tqdm(enumerate(train_dataloader), desc="Batch"):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            logger.info('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    logger.info("")
    logger.info("  Average training loss: {0:.2f}".format(avg_train_loss))
    logger.info("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ------------------------------------------------------------------------------------------------------------------
    # Todo: Cut code from here to remove the validation step: the loading function has to be changed in order
    #  to parse the training set only
    # ------------------------------------------------------------------------------------------------------------------
      # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    logger.info("")
    logger.info("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU/CPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

       # Store gold labels single list
        gold_labels.extend(labels_flat)
        # Store predicted labels single list
        predicted_labels.extend(pred_flat)

        # The classification report is printed on the log, note that print one report for each validation epoch,
        # if we want to compute an average P/R/F1 we can do the same as accuracy, that is an accumulator that
        # stores P/R over epochs or compute the average at the end

        logger.info(classification_report(labels_flat,pred_flat, digits=4))

    # ------------------------------------------------------------------------------------------------------------------
    # Todo: Cut code until here to remove the validation step
    # ------------------------------------------------------------------------------------------------------------------

    # Report the final accuracy for this validation run.
    logger.info("  Accuracy: {0:.2f}".format(eval_accuracy / nb_eval_steps))
    logger.info("  Validation took: {:}".format(format_time(time.time() - t0)))


    logger.info("")
    logger.info("Evaluation on full prediction per epoch!")
    logger.info("Gold labels" + str(len(gold_labels)))
    logger.info("Predicted labels" + str(len(predicted_labels)))
    logger.info(classification_report(gold_labels,predicted_labels, digits=4))

Training:   0%|                                                                                  | 0/5 [00:00<?, ?it/s]2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  
2022-09-14 13:46:16,513 [MainThread  ] [INFO ]  

In [2268]:
logger.info("")
logger.info("Training complete!")

logger.info("Saving model to: " + output_model_dir)
logger.info("# Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()")
# Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = (
    model.module if hasattr(model, "module") else model
)  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,387 [MainThread  ] [INFO ]  
2022-09-14 13:46:55,

('C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\HateBert_models\\RoastMe_Balanced\\1\\tokenizer_config.json',
 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\HateBert_models\\RoastMe_Balanced\\1\\special_tokens_map.json',
 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\HateBert_models\\RoastMe_Balanced\\1\\vocab.txt',
 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\HateBert_models\\RoastMe_Balanced\\1\\added_tokens.json')

In [2269]:
def load_unlabeled_data(tokenizer, test_file_path):

    '''
    Function to load unlabeled data. The input format is one tweet per line:
    tweet_id , tweet text , labels
    :param tokenizer: BERT tokenizer, output of the training code
    :return: the list of
        test_input_ids, test_attention_masks, test_tweet_ids
        which stand for the list of tokenized tweet texts, the list of attention masks, and the list of input tweet ids respectively
        note that the list test_tweet_ids will be used for prediction output
    '''
    # List of all tweets text
    test_tweets = []
    test_tweet_ids = []
    test_labels = []
    # Test Set
    with open(test_file_path, encoding="utf8") as input_file:
        reader = csv.reader(input_file, delimiter=",")
        # For each tweet
        # for line in csv.reader(input_file, delimiter="\t"):
        for line in reader:
        #    line = line.split(",")
            if line[0] != 'id':
                full_line = line[1]
                full_line = text_preproc(full_line)
#                 full_line = re.sub(r'#([^ ]*)', r'\1', full_line)
#                 full_line = re.sub(r'https.*[^ ]', 'URL', full_line)
#                 full_line = re.sub(r'http.*[^ ]', 'URL', full_line)
#                 full_line = re.sub(r'@([^ ]*)', '@USER', full_line)
#                 full_line = emoji.demojize(full_line)
#                 full_line = re.sub(r'(:.*?:)', r' \1 ', full_line)
#                 full_line = re.sub(' +', ' ', full_line)

                # Save tweet's text and label
                test_tweets.append(full_line)
                test_tweet_ids.append(line[0])
                test_labels.append(line[2])

    # List of all tokenized tweets
    test_input_ids = []

    # For every tweet in the test set
    for sent in test_tweets:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=100,

            # This function also supports truncation and conversion
            # to pytorch tensors, but we need to do padding, so we
            # can't use these features :( .
            # max_length = 128,          # Truncate all sentences.
            # return_tensors = 'pt',     # Return pytorch tensors.
        )

        # Add the encoded tweet to the list.
        test_input_ids.append(encoded_sent)

    # # Pad our input tokens with value 0.
    # # "post" indicates that we want to pad and truncate at the end of the sequence,
    # # as opposed to the beginning.
    test_input_ids = pad_sequences(test_input_ids, maxlen=100, dtype="long",
                                    value=tokenizer.pad_token_id, truncating="pre", padding="pre")

    # Create attention masks
    # The attention mask simply makes it explicit which tokens are actual words versus which are padding
    test_attention_masks = []

    # For each tweet in the test set
    for sent in test_input_ids:
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        test_attention_masks.append(att_mask)

    # Return the list of encoded tweets, the list of labels and the list of attention masks
    return test_input_ids, test_attention_masks, test_tweet_ids, test_labels,test_tweets

In [2270]:
dataset_names = ['Kaggle',
                 'Founta',
                 'Razavi',
                'Waseem',
               'Kumar',
                  'RoastMe',
#                  'RoastMe_Data_updated',
                'Offensive_Reddit_new'
                ]
train_dataset = ['Kaggle']
dataset_number = 1
for dataset_name in [name for name in dataset_names if name not in train_dataset]:
#for dataset_name in train_dataset:
    print('test on ' + dataset_name +':\n')
    #RoastMe_new_Data_balanced
    test_file_path = 'RoastMe_new_Data_balanced.csv'
    #test_file_path = 'train_dataset_3_'+dataset_name+'.csv'
    #test_file_path = 'train_dataset_' + str(dataset_number) + '_' +dataset_name+'_imbalanced.csv'
    output_file = 'output.csv'
    prediction_inputs, prediction_masks, tweet_ids, tweet_labels, test_tweets = load_unlabeled_data(tokenizer, test_file_path)
    #print(tweet_labels)
    # Tweets
    prediction_inputs = torch.tensor(prediction_inputs)
    # Attention masks
    prediction_masks = torch.tensor(prediction_masks)
    label_encoder = preprocessing.LabelEncoder()
    targets = label_encoder.fit_transform(tweet_ids)
    # targets: array([0, 1, 2, 3])
    prediction_ids = torch.as_tensor(targets)
    # targets: tensor([0, 1, 2, 3])
    # Set the batch size.
    batch_size = 32
    # Create the DataLoader.
    prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_ids)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
    print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
    # Put model in evaluation mode
    model_to_save.eval()
    # Tracking variables
    predictions = []
    scores = []

    # Predict
    for batch in prediction_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_tweet_ids = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up prediction
        with torch.no_grad():
        # Forward pass, calculate logit predictions
            outputs = model_to_save(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        flat_logits = np.argmax(logits, axis=1).flatten()
        # Get tweet ids for prediction output
        ids = label_encoder.inverse_transform(b_tweet_ids.cpu().numpy())
        # Store predictions and true labels
        predictions.extend(list(zip(ids, flat_logits)))
        scores.extend(list(zip(ids, logits)))
    # Print the list of prediction & store for evaluation
    with open(output_file, 'w', encoding="utf-8") as out_file:
        # store predictions in list for eval
        pred_ = []
        # Get each tweet id
        for tweet_id_prediction in predictions:
            # Print the prediction todo: debug to remove
            #print(str(tweet_id_prediction[0]) + "\t" + str(tweet_id_prediction[1]))
            pred_.append(str(tweet_id_prediction[1]))
            #pred_.append(str(tweet_id_prediction[1]).replace('0', 'NOT').replace('1','ABU'))
            # Append the tweet id along with predicted label on file
            label = '1'
            if str(tweet_id_prediction[1]) == '0':
                label = '0'
    #            pred_.append(label)
    #        pred_.append(label)
            out_file.write(str(tweet_id_prediction[0]) + "," + label + '\n')

    logger.info(len(pred_))
    logger.info(len(tweet_labels))
    logger.info(classification_report(tweet_labels,pred_, digits=4))

    Report = classification_report(tweet_labels, pred_,digits=4, output_dict=True )

    FP = []
    FN = []
    TN = []
    TP = []
    for i in range(len(tweet_labels)):
        if tweet_labels[i] == '0' and  pred_[i] =='1':
            FP.append(test_tweets[i])
        elif tweet_labels[i] == '1' and  pred_[i] =='0':
            FN.append(test_tweets[i])
        elif tweet_labels[i] == '0' and  pred_[i] =='0':
            TN.append(test_tweets[i])
        elif tweet_labels[i] == '1' and  pred_[i] =='1':
            TP.append(test_tweets[i]) 
            
    confusion_matrix = {'Labels':['1','0'],
                   '0':[len(TN),len(FN)],
                   '1':[len(FP),len(TP)]}
    confusion_matrix = pd.DataFrame(confusion_matrix)
    print(confusion_matrix)
            
    dict_HateBert_Test_dataset = {}
    dict_HateBert_Test_dataset['f1_1'] = Report['1']['f1-score']
    dict_HateBert_Test_dataset['f1_weighted'] = Report['weighted avg']['f1-score']
    dict_HateBert_Test_dataset['confusion_matrix'] = confusion_matrix
    dict_HateBert_Test_dataset['FP'] = FP
    dict_HateBert_Test_dataset['FN'] = FN
    dict_HateBert_Test_dataset['TN'] = TN
    dict_HateBert_Test_dataset['TP'] = TP
    print('f1 score:'+str(dict_HateBert_Test_dataset['f1_1']))
    print('f1 weighted score:'+str(dict_HateBert_Test_dataset['f1_weighted']))
    print("5 False Positives: {}".format(dict_HateBert_Test_dataset['FP'][0:6]))
    print("5 False Negatives: {}".format(dict_HateBert_Test_dataset['FN'][0:6]))
    print("5 True Positives: {}".format(dict_HateBert_Test_dataset['TP'][0:6]))
    print("5 True Negatives: {}".format(dict_HateBert_Test_dataset['TN'][0:6]))
    with open( 'dict_HateBert_'+train_dataset[0] +'_on' + dataset_name+'_balanced.pickle', 'wb') as handle:
        pickle.dump(dict_HateBert_Test_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('    DONE.')

test on RoastMe:

Predicting labels for 2,000 test sentences...


2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]  2000
2022-09-14 13:46:58,834 [MainThread  ] [INFO ]

  Labels    0    1
0      1  828  172
1      0  106  894
f1 score:0.8654404646660213
f1 weighted score:0.8608484639772713
5 False Positives: ['tim pool sargon stefan molyneux tim gives honest news sargon wonderful asshole stefan cites explains positions well oh rageaholic fun completely biased enjoy rants ', 'also inanimate object restored like blew up ever jail find always people never experienced system advocate penalties hugely proportion crime punishment harsh enough gain beneficial effects instilling fear would be vandals long enough affect psyches incentivize really think actions reform ways there nothing gained increased harshness punishment negatives like resentment unless course execute lock away forever there risk fucking up course also miss potential heartening effects seeing fuck ups choose reform whatever potential benefits reformed fuck ups may offer world ', 'whats difference battery assault ', 'similar story twitter members regularly search anyone criticising digra hara

### Filtering Dataset By GELU score - Ran on the model that finetuned on Kaggle

In [2271]:
np.argmax(scores[5][1]).flatten()

In [2272]:
scores_df = pd.DataFrame(scores)
scores_df.columns = ['index', 'scores']


In [2273]:
scores_df_new = pd.DataFrame(columns=['score_0','score_1'])
scores_df_new

In [2274]:
scores_0 = []
scores_1 = []
for i in range(len(scores_df)):
    scores_0.append(scores_df['scores'][i][0])
    scores_1.append(scores_df['scores'][i][1])
scores_df['score_0'] = scores_0
scores_df['score_1'] = scores_1
scores_df['True_label'] = tweet_labels
scores_df['predicted'] = pred_
scores_df['Tweet'] = test_tweets
    

#### 

In [2275]:
scores_df

In [2276]:
with open( 'Scores_table_HateBert_finetuned_Kaggle_tested_on_RoastMe.pickle', 'wb') as handle:
    pickle.dump(scores_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2277]:
Best_Positive_RM = scores_df[scores_df['True_label']=='1'].sort_values(by=['score_1'], ascending=False).head(1000).reset_index()[['Tweet','True_label']]

In [2278]:
Best_Negative_RM = scores_df[scores_df['True_label']=='0'].sort_values(by=['score_0'], ascending=False).head(1000).reset_index()[['Tweet','True_label']]

In [2279]:
Roast_Me_dataset_new = pd.concat([Best_Positive_RM,Best_Negative_RM], axis=0, ignore_index=True)

In [2280]:
Roast_Me_dataset_new = Roast_Me_dataset_new.sample(frac=1)

In [2281]:
Roast_Me_dataset_new = Roast_Me_dataset_new.reset_index(drop=True)

In [2282]:
Roast_Me_dataset_new['id'] = [i for i in range(1,len(Roast_Me_dataset_new)+1)]

In [2283]:
Roast_Me_dataset_new = Roast_Me_dataset_new.rename(columns = {"Tweet":"comment","True_label":"Label"})

In [2284]:
Roast_Me_dataset_new =   Roast_Me_dataset_new[['id','comment','Label']]

In [2285]:
Roast_Me_dataset_new['Label'] = Roast_Me_dataset_new['Label'].astype(str)

In [2286]:
Roast_Me_dataset_new

In [2287]:
Roast_Me_dataset_new.to_csv('Roast_me_balanced_new.csv', index=False)

In [2288]:
#os.getcwd()